importing modules

In [ ]:
import pandas as pd
import numpy as np
from nltk.sentiment import SentimentIntensityAnalyzer
from rake_nltk import Rake
import spacy
import matplotlib.pyplot as plt
from gensim.models import Word2Vec
import gensim
import os
import pandas as pd
from sentence_transformers import SentenceTransformer
import re
import preprocessor as p
from sklearn.cluster import KMeans
from wordcloud import WordCloud
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords

In [ ]:
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')

In [ ]:
df= pd.read_csv("Citadines OMR Chennai.csv")

In [4]:
new_names = {
    'a3332d346a': 'Reviewer Name',
    'afac1f68d9': 'Reviewer Country',
    'abf093bdfe': 'Room Type',
    'abf093bdfe 2':'Length of Stay',
    'abf093bdfe 3':'Review Date',
    'abf093bdfe 4':'Traveler Type',
    'abf093bdfe 5':'Review Date',
    'f6431b446c' :'Overall Rating',
    'a53cbfa6de' :'Positive Comments',
    'a53cbfa6de 2 ':'Negative Comments',
    'a3332d346a 2': 'Hotel Response',
    'a53cbfa6de 2': 'negative review',
    'a53cbfa6de 3':'Hotel Response1'
}

# Rename columns
df.rename(columns=new_names, inplace=True)


In [ ]:
df.head(5)

In [6]:
new_df = df['Positive Comments']

In [7]:
new_df = df['Positive Comments'].copy()
new_df.head(5)

0    House keeping people's are awesome 👌 and staff...
1                  The way the apartment was designed.
2    It was clean. It was within short distances to...
3    Excellent Room. Tasteful. Good working area. G...
4                           I like the room very much.
Name: Positive Comments, dtype: object

In [8]:
new_df.head()

0    House keeping people's are awesome 👌 and staff...
1                  The way the apartment was designed.
2    It was clean. It was within short distances to...
3    Excellent Room. Tasteful. Good working area. G...
4                           I like the room very much.
Name: Positive Comments, dtype: object

In [9]:
def clean_tweets(df):
    #set up punctuations we want to be replaced
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\|)|(\()|(\))|(\[)|(\])|(\%)|(\$)|(\>)|(\<)|(\{)|(\})")
    REPLACE_WITH_SPACE = re.compile("(<br\s/><br\s/?)|(-)|(/)|(:).")
    tempArr = []
    for line in df:
        # send to tweet_processor
        tmpL = p.clean(line)
        # remove puctuation
        tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower()) # convert all tweets to lower cases
        tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
        tempArr.append(tmpL)
    return tempArr

In [10]:
df['Positive Comments'].isnull().sum()

76

In [11]:
# Drop rows with missing values in the 'Positive Comments' column
df.dropna(subset=['Positive Comments'], inplace=True)

# Reset the index after dropping rows
df.reset_index(drop=True, inplace=True)

# Print the updated DataFrame
print(len(df['Positive Comments']))


390


In [12]:
df['Positive Comments'] = clean_tweets(df['Positive Comments'])
df.head()
corpus = list(df['Positive Comments'])
corpus = corpus[0:100]
# Filter out non-string elements from the corpus list
corpus = [str(sentence) for sentence in corpus if isinstance(sentence, str)]

# Encode the corpus using the SentenceTransformer model
corpus_embeddings = embedder.encode(corpus)

#corpus_embeddings = embedder.encode(corpus)

In [13]:
corpus_embeddings

array([[-0.14843765, -0.1592571 ,  0.6816438 , ...,  0.6453806 ,
         0.63670355, -0.69390076],
       [ 0.18183896, -0.23908514, -0.14033248, ..., -0.10603566,
         0.5042884 , -0.31477824],
       [-0.34954244, -0.09549483,  0.5599287 , ...,  1.1999667 ,
         0.3542567 , -0.24864605],
       ...,
       [-0.38277772, -0.8140257 ,  0.4645711 , ..., -0.19544713,
         0.7136461 , -0.43428904],
       [ 0.6997522 , -0.4388508 ,  1.0862131 , ...,  0.4909583 ,
        -0.49598986, -0.58794254],
       [-0.43766084, -0.26427838,  0.4162294 , ...,  0.6004491 ,
        -0.11035033, -0.5438475 ]], dtype=float32)

In [14]:
num_clusters = 15
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(corpus_embeddings)
cluster_assignment = clustering_model.labels_

In [15]:
cluster_assignment

array([ 0, 10,  4,  0, 11,  3,  0,  7,  9,  1,  2,  4,  0,  6,  7,  6,  6,
        1,  0,  6,  0, 12,  2,  6, 10, 10, 14, 12,  2,  5,  2,  5,  5,  3,
        2, 13,  2,  3, 14,  6,  0,  2, 11, 13,  7,  9,  5,  9, 14,  5,  3,
        7, 12,  0, 12, 12,  3,  0,  4, 10,  5,  6,  6,  3,  4,  3,  1, 12,
        9,  6, 12,  0,  8, 11,  3, 12, 12,  9,  3, 13,  0,  6, 13, 14,  4,
        2,  1,  3, 12,  1,  6, 13, 14,  3, 12,  1,  3,  3, 14,  5])

In [16]:
cluster_df = pd.DataFrame(corpus, columns = ['corpus'])
cluster_df['cluster'] = cluster_assignment
cluster_df.head()

,corpus,cluster
0,house keeping peoples are awesome and staff ar...,0
1,the way the apartment was designed,10
2,it was clean it was within short distances to ...,4
3,excellent room tasteful good working area grea...,0
4,i like the room very much,11


In [17]:
clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

for i, cluster in enumerate(clustered_sentences):
    print("Cluster ", i+1)
    print(cluster)
    print("")

Cluster  1
['house keeping peoples are awesome and staff are very friendly', 'excellent room tasteful good working area great staff', 'overall the property was really good well maintained staff was helpful  rooms were spacious and in good conditionbreakfast was good too', 'the serenity at the property was good the staff were courteous', 'the staff was very courteous and responsive the room was quite comfortable', 'very helpful staff clean and spaciousfood was very good', 'comfortable and calm friendly staff good house keeping', 'excellent location nice spacious room and apartment good customer service', 'nice rooms well maintained  has the good services and facilities', 'room was clean and the house keeper were responding politely and other employees are also behaved well', 'some staffs were really good the room']

Cluster  2
['the staff at reception was really very helpful especially is a true service champ she took all effort to get our previous bookings and bills  thanks a lot', 'on

In [18]:
df['Positive Comments']

0      house keeping peoples are awesome and staff ar...
1                     the way the apartment was designed
2      it was clean it was within short distances to ...
3      excellent room tasteful good working area grea...
4                              i like the room very much
                             ...                        
385                                               normal
386              excellent room no disturbance good food
387                   the rooms were well made and clean
388                                        main location
389                                              nothing
Name: Positive Comments, Length: 390, dtype: object

In [19]:

df['Positive Comments'] = df['Positive Comments'].drop_duplicates()

In [33]:
# Assuming `cluster_assignment` contains the cluster assignment for each sentence
# and `corpus` contains all the sentences

num_clusters = len(set(cluster_assignment))  # Get the number of clusters

clustered_sentences = [[] for _ in range(num_clusters)]

for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(corpus[sentence_id])

# Print the clustered sentences
for i, cluster in enumerate(clustered_sentences):
    print(f"Cluster {i+1}:")
    for sentence in cluster:
        print(sentence)
    print("")


Cluster 1:
house keeping peoples are awesome and staff are very friendly
excellent room tasteful good working area great staff
overall the property was really good well maintained staff was helpful  rooms were spacious and in good conditionbreakfast was good too
the serenity at the property was good the staff were courteous
the staff was very courteous and responsive the room was quite comfortable
very helpful staff clean and spaciousfood was very good
comfortable and calm friendly staff good house keeping
excellent location nice spacious room and apartment good customer service
nice rooms well maintained  has the good services and facilities
room was clean and the house keeper were responding politely and other employees are also behaved well
some staffs were really good the room

Cluster 2:
the staff at reception was really very helpful especially is a true service champ she took all effort to get our previous bookings and bills  thanks a lot
only thing we have carry the room card ev

In [34]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords



# Define a function to extract the most frequent words from a list of sentences
def extract_keywords(sentences):
    # Tokenize the sentences into words
    words = [word.lower() for sentence in sentences for word in word_tokenize(sentence)]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Calculate the frequency distribution of words
    freq_dist = FreqDist(words)

    # Get the most common words (you can adjust the number as needed)
    most_common_words = freq_dist.most_common(3)

    # Extract the words from the tuples and join them into a single string
    keywords = ' '.join([word for word, _ in most_common_words])

    return keywords

# Assuming `clustered_sentences` contains the clustered sentences
# Iterate over each cluster and extract keywords to use as the cluster title
for i, cluster in enumerate(clustered_sentences):
    keywords = extract_keywords(cluster)
    print(f"Cluster {i+1} Title: {keywords}")


Cluster 1 Title: good staff room
Cluster 2 Title: clean hotel room
Cluster 3 Title: cleanliness staff friendly
Cluster 4 Title: room clean location
Cluster 5 Title: staff stay room
Cluster 6 Title: ambience location ambiance
Cluster 7 Title: good breakfast food
Cluster 8 Title: good room nice
Cluster 9 Title: sauna work
Cluster 10 Title: service order good
Cluster 11 Title: location way apartment
Cluster 12 Title: like room much
Cluster 13 Title: good everything excellent
Cluster 14 Title: good room comfortable
Cluster 15 Title: pool swimming gym


In [35]:
import json
import pandas as pd

# Assuming `clustered_sentences` contains the clustered sentences
cluster_titles = {}

# Iterate over each cluster and extract keywords to use as the cluster title
for i, cluster in enumerate(clustered_sentences):
    keywords = extract_keywords(cluster)
    cluster_titles[f"Cluster {i+1}"] = {'title': keywords, 'strength': len(cluster)}

# Convert the cluster titles dictionary to a DataFrame
df = pd.DataFrame.from_dict(cluster_titles, orient='index')

# Rename the index to match the column name
df.index.name = 'Citadines OMR Chennai hotel strengths'

# Save the DataFrame as JSON
df.to_json('cluster_titles.json', indent=4)


In [38]:
df.head(10)

,title,strength
Citadines OMR Chennai hotel strengths,,
Cluster 1,good staff room,11
Cluster 2,clean hotel room,6
Cluster 3,cleanliness staff friendly,8
Cluster 4,room clean location,13
Cluster 5,staff stay room,5
Cluster 6,ambience location ambiance,7
Cluster 7,good breakfast food,11
Cluster 8,good room nice,4
Cluster 9,sauna work,1


weekness


In [24]:
ndf = df['Hotel Response1']

In [25]:
ndf.isnull().sum()

105

In [26]:
len(ndf)

390

In [27]:
df['Hotel Response1'] = df['Hotel Response1'].drop_duplicates()

In [28]:
df.head()

,Reviewer Name,Reviewer Country,Room Type,Length of Stay,Review Date,Traveler Type,Review Date,Overall Rating,Positive Comments,Hotel Response,negative review,Hotel Response1
0,Saiful,Bangladesh,Studio Executive,4 nights ·,March 2023,Family,Reviewed: 11 April 2023,Exceptional,house keeping peoples are awesome and staff ar...,Property response:,Thank you for taking the time out of your busy...,NaN
1,Nirudhya,Australia,Family,One-Bedroom Apartment,March 2024,16 nights ·,Reviewed: 1 April 2024,Exceptional. Staff were very friendly and faci...,the way the apartment was designed,Property response:,Thank you for taking the time out of your busy...,The location as there was a lot of noise from ...
2,R,Singapore,Couple,Studio Deluxe,March 2024,3 nights ·,Reviewed: 1 April 2024,Would love to go back and stay here😊,it was clean it was within short distances to ...,Property response:,Thank you for sharing your experience at Citad...,I was not informed that the key card needs to ...
3,Karthik,United States,Solo traveller,One Bedroom Executive,March 2024,3 nights ·,Reviewed: 31 March 2024,I will return for the excellent tasteful and s...,excellent room tasteful good working area grea...,Property response:,"On behalf of the entire team, I would like to ...","The Wi-Fi was good in the room, but it was pat..."
4,Raja,India,Couple,Studio Deluxe,March 2024,1 night ·,Reviewed: 19 March 2024,It was excellent,i like the room very much,Property response:,Thank you for rating us!\nWarm Regards \nThe M...,Closing of the pool entrance


In [29]:
len(df)

390

In [ ]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
import preprocessor as p
import re
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')
def clean_tweets(df):
    REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    tempArr = []
    for line in df:
        # Check if the value is NaN
        if pd.isnull(line):
            tempArr.append("")
            continue
        # Send to tweet_processor
        tmpL = p.clean(line)
        # Remove punctuation
        tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower())
        # Replace specific characters with spaces
        tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
        # Remove extra spaces
        tmpL = " ".join(tmpL.split())
        tempArr.append(tmpL)
    return tempArr


def cluster_sentences(df, column_name, num_clusters=15):
    # Clean the tweets
    df[column_name] = clean_tweets(df[column_name])
    
    # Get a subset of the corpus
    corpus = list(df[column_name])[0:100]
    corpus = [str(sentence) for sentence in corpus if isinstance(sentence, str)]
    
    # Encode the corpus using the SentenceTransformer model
    corpus_embeddings = embedder.encode(corpus)
    
    # Perform KMeans clustering
    clustering_model = KMeans(n_clusters=num_clusters)
    clustering_model.fit(corpus_embeddings)
    cluster_assignment = clustering_model.labels_
    
    # Create a DataFrame with the clustered sentences
    cluster_df = pd.DataFrame({'corpus': corpus, 'cluster': cluster_assignment})
    
    # Return the clustered DataFrame
    return cluster_df

def extract_keywords(sentences):
    # Tokenize the sentences into words
    words = [word.lower() for sentence in sentences for word in word_tokenize(sentence)]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Calculate the frequency distribution of words
    freq_dist = FreqDist(words)

    # Get the most common words (you can adjust the number as needed)
    most_common_words = freq_dist.most_common(3)

    # Extract the words from the tuples and join them into a single string
    keywords = ' '.join([word for word, _ in most_common_words])

    return keywords

def get_cluster_titles(clustered_sentences):
    # Iterate over each cluster and extract keywords to use as the cluster title
    cluster_titles = []
    for cluster in clustered_sentences:
        keywords = extract_keywords(cluster)
        cluster_titles.append(keywords)
    
    return cluster_titles

# Assuming df is your DataFrame and 'Positive Comments' is the column containing the sentences
cluster_df = cluster_sentences(df, 'Hotel Response1')
clustered_sentences = [list(cluster_df[cluster_df['cluster'] == i]['corpus']) for i in range(num_clusters)]
cluster_titles = get_cluster_titles(clustered_sentences)

# Print the clustered sentences and their titles


In [31]:
for i, (cluster, title) in enumerate(zip(clustered_sentences, cluster_titles)):
    print(f"Cluster {i+1} Title: {title}")
    print("")


Cluster 1 Title: room mobile hotel

Cluster 2 Title: na wlan nil

Cluster 3 Title: late response

Cluster 4 Title: staffs lesser natural

Cluster 5 Title: could reduce price

Cluster 6 Title: nothing think non

Cluster 7 Title: room size small

Cluster 8 Title: location lot noise

Cluster 9 Title: us luggage told

Cluster 10 Title: bit could front

Cluster 11 Title: staff might need

Cluster 12 Title: nothing tv bad

Cluster 13 Title: overpriced cost increased

Cluster 14 Title: breakfast closing pool

Cluster 15 Title: check waiting time



In [3]:
df=pd.read_csv("Clarion Hotel President.csv")
new_names = {
    'a3332d346a': 'Reviewer Name',
    'afac1f68d9': 'Reviewer Country',
    'abf093bdfe': 'Room Type',
    'abf093bdfe 2':'Length of Stay',
    'abf093bdfe 3':'Review Date',
    'abf093bdfe 4':'Traveler Type',
    'abf093bdfe 5':'Review Date',
    'f6431b446c' :'Overall Rating',
    'a53cbfa6de' :'Positive Comments',
    'a53cbfa6de 2 ':'Negative Comments',
    'a3332d346a 2': 'Hotel Response',
    'a53cbfa6de 2': 'negative review',
    'a53cbfa6de 3':'Hotel Response1'
}

# Rename columns
df.rename(columns=new_names, inplace=True)


In [4]:
df.head(5)

,Reviewer Name,Reviewer Country,Room Type,Length of Stay,Review Date,Traveler Type,Review Date,Overall Rating,Positive Comments,negative review
0,Peter,Australia,Deluxe Double or Twin Room,2 nights ·,November 2023,Solo traveller,Reviewed: 25 November 2023,Exceptional,Fantastic and friendly staff - from the person...,NaN
1,Indranil,India,Deluxe Double or Twin Room,6 nights ·,March 2024,Solo traveller,Reviewed: 30 March 2024,Good,Breakfast and the room,Rules
2,Stephanos,Cyprus,Couple,Superior King Room,February 2024,1 night ·,Reviewed: 19 February 2024,Staff were wonderful - the receptionist (Iqb...,room breakfast and location were all great,There were problems to register and access wif...
3,Premanand,India,Deluxe Double or Twin Room,3 nights ·,September 2023,Family,Reviewed: 28 November 2023,Passable,Late Dinner\nNo Proper response from Cooking t...,Late Dinner\nNo Proper response from Cooking t...
4,Raymond,United Kingdom,Deluxe Double or Twin Room,3 nights ·,November 2023,Family,Reviewed: 13 November 2023,High expectations but not delivered 😕,Spacious room with a great view and a comforta...,False advertisement of alcohol facility - the ...


In [6]:
import pandas as pd
from textblob import TextBlob

df_n=pd.DataFrame(df['negative review'])
df_n = df_n.dropna()

# Remove duplicates
df_n = df_n.drop_duplicates()

# If you want to reset the index after removing rows, you can use reset_index
df_n = df_n.reset_index(drop=True)
# Function to analyze sentiment
def analyze_sentiment(text):
    blob = TextBlob(text)
    polarity = blob.sentiment.polarity
    if polarity > 0:
        return 'Positive'
    elif polarity < 0:
        return 'Negative'
    else:
        return 'Neutral'

# Apply sentiment analysis to the DataFrame column
df_n['sentiment'] = df_n['negative review'].apply(lambda x: analyze_sentiment(x))



In [7]:
df_n

,negative review,sentiment
0,Rules,Neutral
1,There were problems to register and access wif...,Negative
2,Late Dinner\nNo Proper response from Cooking t...,Negative
3,False advertisement of alcohol facility - the ...,Negative
4,nothing,Neutral
...,...,...
332,Pas de wifi stable dans les chambres.,Neutral
333,- Maus in der Lobby\n- Personal\n- Etwas abgen...,Neutral
334,Falta de mantenimiento,Neutral
335,la propreté de la salle de bains,Neutral


In [9]:
df_n['sentiment'].value_counts()

sentiment
Negative    145
Positive     98
Neutral      94
Name: count, dtype: int64

In [10]:
import pandas as pd
from textblob import TextBlob

def analyze_sentiment_df(df, text_column_name):
    df_text = pd.DataFrame(df[text_column_name])
    df_text = df_text.dropna()
    df_text = df_text.drop_duplicates()
    df_text = df_text.reset_index(drop=True)

    def analyze_sentiment(text):
        blob = TextBlob(text)
        polarity = blob.sentiment.polarity
        if polarity > 0:
            return 'Positive'
        elif polarity < 0:
            return 'Negative'
        else:
            return 'Neutral'

    df_text['sentiment'] = df_text[text_column_name].apply(lambda x: analyze_sentiment(x))
    return df_text

# Example usage
# Assuming `df` is your DataFrame and 'negative review' is the column containing the text
df_sentiment = analyze_sentiment_df(df, 'negative review')


In [11]:
df_sentiment

,negative review,sentiment
0,Rules,Neutral
1,There were problems to register and access wif...,Negative
2,Late Dinner\nNo Proper response from Cooking t...,Negative
3,False advertisement of alcohol facility - the ...,Negative
4,nothing,Neutral
...,...,...
332,Pas de wifi stable dans les chambres.,Neutral
333,- Maus in der Lobby\n- Personal\n- Etwas abgen...,Neutral
334,Falta de mantenimiento,Neutral
335,la propreté de la salle de bains,Neutral


In [8]:
!streamlit run a.py

^C


In [13]:
import os
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sentence_transformers import SentenceTransformer
import preprocessor as p
import re
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
import json

# Initialize the SentenceTransformer model
embedder = SentenceTransformer('distilbert-base-nli-mean-tokens')

# Define a function to clean tweets
def clean_tweets(df):
    REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    tempArr = []
    for line in df:
        # Check if the value is NaN
        if pd.isnull(line):
            tempArr.append("")
            continue
        # Send to tweet_processor
        tmpL = p.clean(line)
        # Remove punctuation
        tmpL = REPLACE_NO_SPACE.sub("", tmpL.lower())
        # Replace specific characters with spaces
        tmpL = REPLACE_WITH_SPACE.sub(" ", tmpL)
        # Remove extra spaces
        tmpL = " ".join(tmpL.split())
        tempArr.append(tmpL)
    return tempArr

# Function to cluster sentences
def cluster_sentences(df, column_name):
    # Clean the tweets
    df[column_name] = clean_tweets(df[column_name])
    
    # Drop null values
    df = df.dropna()

    # Remove duplicates
    df = df.drop_duplicates()

    # Reset index
    df = df.reset_index(drop=True)
    
    # Get a subset of the corpus
    corpus = list(df[column_name])
    corpus = [str(sentence) for sentence in corpus if isinstance(sentence, str)]
    
    # Encode the corpus using the SentenceTransformer model
    corpus_embeddings = embedder.encode(corpus)
    
    # Find the optimal number of clusters
    max_clusters = min(len(corpus), 15)  # Maximum number of clusters
    best_score = -1
    best_clusters = 2
    for n_clusters in range(2, max_clusters + 1):
        kmeans = KMeans(n_clusters=n_clusters)
        cluster_labels = kmeans.fit_predict(corpus_embeddings)
        silhouette_avg = silhouette_score(corpus_embeddings, cluster_labels)
        if silhouette_avg > best_score:
            best_score = silhouette_avg
            best_clusters = n_clusters
    
    # Ensure a minimum of 5 clusters
    best_clusters = max(best_clusters, 5)
    
    # Perform KMeans clustering with the optimal number of clusters
    clustering_model = KMeans(n_clusters=best_clusters)
    clustering_model.fit(corpus_embeddings)
    cluster_assignment = clustering_model.labels_
    
    # Create a DataFrame with the clustered sentences
    cluster_df = pd.DataFrame({'corpus': corpus, 'cluster': cluster_assignment})
    
    # Return the clustered DataFrame
    return cluster_df, best_clusters

# Function to extract keywords from sentences
def extract_keywords(sentences):
    # Tokenize the sentences into words
    words = [word.lower() for sentence in sentences for word in word_tokenize(sentence)]

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    # Calculate the frequency distribution of words
    freq_dist = FreqDist(words)

    # Get the most common words (you can adjust the number as needed)
    most_common_words = freq_dist.most_common(3)

    # Extract the words from the tuples and join them into a single string
    keywords = ' '.join([word for word, _ in most_common_words])

    return keywords

# Clean column names to ensure uniqueness
def clean_column_names(df):
    # Ensure all column names are unique
    cols = pd.Series(df.columns)
    for dup in cols[cols.duplicated()].unique():
        cols[cols[cols == dup].index.values.tolist()] = [dup + '_' + str(i) if i != 0 else dup for i in range(sum(cols == dup))]
    df.columns = cols
    # Clean the column names for consistency
    df.columns = df.columns.map(lambda x: x.strip().lower().replace(' ', '_'))
    return df

# Load all CSV files from the directory
directory = r'D:\21761A4230\preproced hotel data'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)
        df = clean_column_names(df)
        
        # Cluster and extract keywords
        cluster_df, num_clusters = cluster_sentences(df, 'cleaneddata')  # Assuming 'positive_comments' is the column containing positive comments
        clustered_sentences = [list(cluster_df[cluster_df['cluster'] == i]['corpus']) for i in range(num_clusters)]

        # Prepare data for JSON
        data = {}
        for i, cluster in enumerate(clustered_sentences):
            keywords = extract_keywords(cluster)
            data[f"Cluster {i+1}"] = {
                "keywords": keywords,
                "hotels": cluster  # Assuming 'cluster' contains hotel names
            }

        # Save JSON files for each hotel
        output_folder = os.path.splitext(filepath)[0]
        os.makedirs(output_folder, exist_ok=True)
        for hotel_name, hotel_data in data.items():
            output_file = os.path.join(output_folder, f"{hotel_name}_keywords.json")
            with open(output_file, "w") as json_file:
                json.dump(hotel_data, json_file)



In [17]:
!streamlit run a1.py

^C
